<a href="https://colab.research.google.com/github/lehenbauer/colab/blob/main/Copy_of_One_Click_Jukebox_Continuous_Reprime.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#(Almost) One-click Jukebox Continuous Reprime notebook.

Join the Jukebox community at https://discord.gg/aEqXFN9amV

Based on One-Click Jukebox Notebook by Broccaloo: https://www.youtube.com/c/broccaloo/

A bit based on JuicyJukebox's Notebook by Reed Mayhew: https://github.com/reedmayhew18/

Continuous Repriming made by MichaelsLab: https://www.youtube.com/user/CraftMine1000

Explanation about Continuous Reprime: https://youtu.be/BPo5sECkBV4?t=100 and https://youtu.be/BPo5sECkBV4?t=1491

In [ ]:
#@title Check which GPU you were assigned by running this cell.
simple_view = True #@param {type: "boolean"}
if simple_view:
  !nvidia-smi -L
else:
  !nvidia-smi
your_lyrics = """
"""

GPU 0: Tesla T4 (UUID: GPU-bfd6e819-4590-3d72-4dba-99833c932953)


In [ ]:
# Optional, put lyrics here if you copy your lyrics and paste it then run this cell, 
# or else leave it blank and not running this cell.
your_lyrics = """
"""

In [ ]:
#@title Select your settings and click Runtime -> "Run All" or "Restart and Run All" to start repriming. { vertical-output: true, form-width: "32%" }

from google.colab import drive
drive.mount('/content/gdrive')

!pip install --upgrade git+https://github.com/craftmine1000/jukebox-saveopt.git
!apt install ffmpeg sox

import jukebox
import torch as t
import torch.nn.functional as F
import librosa
import os
import soundfile as sf
from IPython.display import Audio
from jukebox.make_models import make_vqvae, make_prior, MODELS, make_model
from jukebox.hparams import Hyperparams, setup_hparams
from jukebox.sample import sample_single_window, _sample, \
                           sample_partial_window, upsample, \
                           load_prompts
from jukebox.utils.dist_utils import setup_dist_from_mpi
from jukebox.utils.torch_utils import empty_cache
# MPI Connect. MPI doesn't like being initialized twice, hence the following
try:
    if device is not None:
        pass
except NameError:
    rank, local_rank, device = setup_dist_from_mpi()

#dir_cache_models = '/root/.cache/jukebox/models'
#!rm -rf $dir_cache_models

model = "5b_lyrics" #@param ["5b_lyrics", "5b", "1b_lyrics"]
if model == '5b':
  your_lyrics = """
  """

save_and_load_models_from_drive = False

#START GDRIVE MODEL LOADER
if save_and_load_models_from_drive == True:
  import os.path
  !apt install pv
  !mkdir /root/.cache ; mkdir /root/.cache/jukebox ; mkdir /root/.cache/jukebox/models ; mkdir /root/.cache/jukebox/models/1b_lyrics ; mkdir /root/.cache/jukebox/models/5b_lyrics ; mkdir /root/.cache/jukebox/models/5b
  !mkdir /content/gdrive/MyDrive/jukebox
  !mkdir /content/gdrive/MyDrive/jukebox/models
  !mkdir /content/gdrive/MyDrive/jukebox/models/5b
  !mkdir /content/gdrive/MyDrive/jukebox/models/5b_lyrics
  !mkdir /content/gdrive/MyDrive/jukebox/models/1b_lyrics


def load_5b_vqvae():
    if os.path.exists("/root/.cache/jukebox/models/5b/vqvae.pth.tar") == False:
      if os.path.exists("/content/gdrive/MyDrive/jukebox/models/5b/vqvae.pth.tar") == False:
        print("5b_vqvae not stored in Google Drive. Downloading for the first time.")
        !wget https://openaipublic.azureedge.net/jukebox/models/5b/vqvae.pth.tar -O /content/gdrive/MyDrive/jukebox/models/5b/vqvae.pth.tar
      else:
        print("5b_vqvae stored in Google Drive.")
      print('Copying 5b VQVAE')
      !pv /content/gdrive/MyDrive/jukebox/models/5b/vqvae.pth.tar > /root/.cache/jukebox/models/5b/vqvae.pth.tar

def load_1b_lyrics_level2():
  if os.path.exists("/root/.cache/jukebox/models/1b_lyrics/prior_level_2.pth.tar") == False:
    if os.path.exists("/content/gdrive/MyDrive/jukebox/models/1b_lyrics/prior_level_2.pth.tar") == False:
      print("1b_lyrics_level_2 not stored in Google Drive. Downloading for the first time. This will take a few more minutes.")
      !wget https://openaipublic.azureedge.net/jukebox/models/1b_lyrics/prior_level_2.pth.tar -O /content/gdrive/MyDrive/jukebox/models/1b_lyrics/prior_level_2.pth.tar
    else:
      print("1b_lyrics_level_2 stored in Google Drive.")
    print("Copying 1B_Lyrics Level 2")
    !pv /content/gdrive/MyDrive/jukebox/models/1b_lyrics/prior_level_2.pth.tar > /root/.cache/jukebox/models/1b_lyrics/prior_level_2.pth.tar

def load_5b_lyrics_level2():
  if os.path.exists("/root/.cache/jukebox/models/5b_lyrics/prior_level_2.pth.tar") == False:
    if os.path.exists("/content/gdrive/MyDrive/jukebox/models/5b_lyrics/prior_level_2.pth.tar") == False:
      print("5b_lyrics_level_2 not stored in Google Drive. Downloading for the first time. This will take up to 10-15 minutes.")
      !wget https://openaipublic.azureedge.net/jukebox/models/5b_lyrics/prior_level_2.pth.tar -O /content/gdrive/MyDrive/jukebox/models/5b_lyrics/prior_level_2.pth.tar
    else:
      print("5b_lyrics_level_2 stored in Google Drive.")
    print("Copying 5B_Lyrics Level 2")
    !pv /content/gdrive/MyDrive/jukebox/models/5b_lyrics/prior_level_2.pth.tar > /root/.cache/jukebox/models/5b_lyrics/prior_level_2.pth.tar

def load_5b_level1():
  if os.path.exists('/root/.cache/jukebox/models/5b/prior_level_1.pth.tar') == False:
    if os.path.exists("/content/gdrive/MyDrive/jukebox/models/5b/prior_level_1.pth.tar") == False:
      print("5b_level_1 not stored in Google Drive. Downloading for the first time. This may take a few more minutes.")
      !wget https://openaipublic.azureedge.net/jukebox/models/5b/prior_level_1.pth.tar -O /content/gdrive/MyDrive/jukebox/models/5b/prior_level_1.pth.tar
    else:
      print("5b_level_1 stored in Google Drive.")
    print("Copying 5B Level 1")
    !pv /content/gdrive/MyDrive/jukebox/models/5b/prior_level_1.pth.tar > /root/.cache/jukebox/models/5b/prior_level_1.pth.tar

def load_5b_level0():
  if os.path.exists('/root/.cache/jukebox/models/5b/prior_level_0.pth.tar') == False:
    if os.path.exists("/content/gdrive/MyDrive/jukebox/models/5b/prior_level_0.pth.tar") == False:
      print("5b_level_0 not stored in Google Drive. Downloading for the first time. This may take a few minutes.")
      !wget https://openaipublic.azureedge.net/jukebox/models/5b/prior_level_0.pth.tar -O /content/gdrive/MyDrive/jukebox/models/5b/prior_level_0.pth.tar
    else:
      print("5b_level_0 stored in Google Drive.")
    print("Copying 5B Level 0")
    !pv /content/gdrive/MyDrive/jukebox/models/5b/prior_level_0.pth.tar > /root/.cache/jukebox/models/5b/prior_level_0.pth.tar

def load_5b_level2():
  if os.path.exists('/root/.cache/jukebox/models/5b/prior_level_2.pth.tar') == False:
    if os.path.exists("/content/gdrive/MyDrive/jukebox/models/5b/prior_level_2.pth.tar") == False:
      print("5b_level_2 not stored in Google Drive. Downloading for the first time. This will take up to 10-15 minutes.")
      !wget https://openaipublic.azureedge.net/jukebox/models/5b/prior_level_2.pth.tar -O /content/gdrive/MyDrive/jukebox/models/5b/prior_level_2.pth.tar
    else:
      print("5b_level_2 stored in Google Drive.")
  print("Copying 5B Level 2")
  !pv /content/gdrive/MyDrive/jukebox/models/5b/prior_level_2.pth.tar > /root/.cache/jukebox/models/5b/prior_level_2.pth.tar

if save_and_load_models_from_drive == True:
  if model == '5b_lyrics':
    load_5b_vqvae()
    load_5b_lyrics_level2()
    load_5b_level1()
    load_5b_level0()
  if model == '5b':
    load_5b_vqvae()
    load_5b_level2()
    load_5b_level1()
    load_5b_level0()
  elif model == '1b_lyrics':
    load_5b_vqvae()
    load_1b_lyrics_level2()
    load_5b_level1()
    load_5b_level0()
#END GDRIVE MODEL LOADER

hps = Hyperparams()
hps.sr = 44100
hps.n_samples = 1
# Specifies the directory to save the sample in.
# We set this to the Google Drive mount point.
#@markdown **Do not append the / at the end of the field, or else it will not work.**
hps.name = '/content/gdrive/MyDrive/03_Jukebox/230331_newjeans-attention_01' #@param {type:"string"}
chunk_size = 128 # if model in ('5b', '5b_lyrics') else 1024
gpu_info = !nvidia-smi -L
if gpu_info[0].find('Tesla T4') >= 0:
  max_batch_size = 2
  print('Tesla T4 detected, max_batch_size set to 2')
elif gpu_info[0].find('Tesla K80') >= 0:
  max_batch_size = 8
  print('Tesla K80 detected, max_batch_size set to 8')
elif gpu_info[0].find('Tesla P100') >= 0:
  max_batch_size = 3
  print('Tesla P100 detected, max_batch_size set to 3')
elif gpu_info[0].find('Tesla V100') >= 0:
  max_batch_size = 3
  print('Tesla V100 detected, max_batch_size set to 3')
elif gpu_info[0].find('A100') >= 0:
  max_batch_size = 6
  print('A100 detected! Congrats!!! max_batch_size set to 6.')
else:
  max_batch_size = 3
  print('Different GPU detected, max_batch_size set to 3.')
hps.levels = 3
speed_upsampling = True #@param {type: "boolean"}
if speed_upsampling == True:
  hps.hop_fraction = [1,1,.125]
else:
  hps.hop_fraction = [.5,.5,.125]

audio_file = '/content/gdrive/MyDrive/03_Jukebox/_audio-samples/01_repriming/newjeans_attention_105bpm.wav' #@param {type: "string"}

os.system('ffmpeg -y -i "'+audio_file+'" -vn -sn -c:a pcm_s24le -ac 2 -ar 44100 /content/pre-primer.wav')
os.system("sox /content/pre-primer.wav -c 1 -b 16 -e signed /content/primer.wav norm -1.2")
!rm /content/pre-primer.wav
audio_file = "/content/primer.wav"

primer_length_in_seconds = librosa.get_duration(filename=audio_file)
duration = (int(primer_length_in_seconds * hps.sr) // 128) * 128
hps.sample_length = duration
raw_audio = load_prompts([audio_file], duration, hps)

try:
  try:
    zs = t.load(f'{hps.name}/level_1/data.pth.tar')['zs']
  except:
    zs = t.load(f'{hps.name}/tokens.t')
except:
  zs = [t.zeros(hps.n_samples,0,dtype=t.long, device='cpu') for _ in range(hps.levels)]

vqvae, *priors = MODELS[model]
vqvae = make_vqvae(setup_hparams(vqvae, dict(sample_length = 1048576)), device)
if zs[-1].shape[1] < duration // 128:
  top_prior = make_prior(setup_hparams(priors[-1], dict()), vqvae, device)
  top_prior.c_to(device)

#@markdown ---

#@markdown Recommended to leave artist/genre on "No Preference" if you are new to Jukebox.

#@markdown If you do set to "Custom", please fill the custom_artist and/or custom_genre field.
#@markdown List of the v2 (5b_lyrics & 5b models) & v3 (1b_lyrics model) artist and genre ids: https://github.com/openai/jukebox/tree/master/jukebox/data/ids

select_artist = "nine inch nails" #@param ["No Preference", "Custom", "112", "*nsync", "10 years", "10cc", "16 volt", "2 chainz", "2 live crew", "2 unlimited", "2pac", "36 crazyfists", "38 special", "4him", "5 seconds of summer", "50 cent", "7 seconds", "a day to remember", "a tribe called quest", "aaron carter", "aaron copland", "aaron neville", "aaron tippin", "aaron watson", "abba", "abbey lincoln", "abc", "abney park", "acappella", "accept", "ace frehley", "ace of base", "acid drinkers", "adam ant", "adam green", "adam lambert", "adam sandler", "adolf busch", "adrian belew", "adriano celentano", "aerosmith", "aesop rock", "after all", "agathocles", "agnostic front", "agonoize", "agoraphobic nosebleed", "aiden", "aimee mann", "air", "air supply", "akon", "al green", "al jarreau", "al martino", "al stewart", "alabama", "alan jackson", "alan stivell", "albert collins", "albert king", "alborosie", "alejandro escovedo", "alesana", "alessandro scarlatti", "alexz johnson", "alfred cortot", "ali project", "alice cooper", "alice in chains", "alicia de larrocha", "alicia keys", "alison krauss", "alison moyet", "alkaline trio", "all", "all saints", "all that remains", "allison moorer", "alpha blondy", "alphaville", "altan", "alvin lee", "america", "amon amarth", "amorphis", "amos lee", "amy grant", "anastacia", "anathema", "and one", "andr previn", "andr s schiff", "andrea bocelli", "andrew lloyd webber", "andrew peterson", "andromeda", "andy partridge", "ane brun", "angel", "angelo branduardi", "angie stone", "angra", "animal collective", "anna ternheim", "anne clark", "anne murray", "anne sophie mutter", "anner bylsma", "annie lennox", "annihilator", "anthony hamilton", "anthony phillips", "anthrax", "anton n dvo k", "antonio meneses", "antonio vivaldi", "antony and the johnsons", "anvil", "aphex twin", "apocalyptica", "apologetix", "april wine", "arabesque", "arcade fire", "arcana", "arcangelo corelli", "architects", "archive", "arena", "aretha franklin", "ariana grande", "ariel pink", "arlo guthrie", "armand van helden", "armin van buuren", "armored saint", "arno", "art garfunkel", "arthur grumiaux", "arthur rubinstein", "artie shaw", "artillery", "artrosis", "artur schnabel", "arturo benedetti michelangeli", "as i lay dying", "ashanti", "asia", "asleep at the wheel", "asp", "asphyx", "assemblage 23", "astrud gilberto", "aswad", "atb", "atmosphere", "atomic kitten", "atreyu", "atrocity", "attila", "audio adrenaline", "august burns red", "autopsy", "avantasia", "avatar", "avenged sevenfold", "aviators", "avicii", "avulsed", "axel rudi pell", "axxis", "ayreon", "az", "b la bart k", "babbie mason", "baby bash", "babyface", "babyshambles", "backstreet boys", "backyard babies", "bad boys blue", "bad company", "bad religion", "badfinger", "badly drawn boy", "bananarama", "barbara mandrell", "barbra streisand", "barclay james harvest", "barry louis polisar", "barry white", "basement jaxx", "basshunter", "bathory", "bauhaus", "bay city rollers", "beach house", "beardfish", "beastie boys", "beat crusaders", "beau", "beaux arts trio", "bebo norman", "beck", "bed ich smetana", "bee gees", "behemoth", "belinda carlisle", "bell x1", "bella morte", "belle and sebastian", "belphegor", "ben lee", "benjamin britten", "benny benassi", "benny goodman", "beres hammond", "berlin", "bert jansch", "bessie smith", "beth hart", "beth nielsen chapman", "beth orton", "bethel music", "bette midler", "between the buried and me", "big audio dynamite", "big big train", "big bill broonzy", "big d and the kids table", "big joe turner", "big sean", "big tymers", "bill anderson", "bill monroe", "bill nelson", "bill withers", "billie holiday", "billy bragg", "billy currington", "billy gilman", "billy idol", "billy joe royal", "billy joe shaver", "billy joel", "billy ocean", "billy preston", "billy ray cyrus", "billy squier", "bing crosby", "biohazard", "birth control", "bizzy bone", "black flag", "black label society", "black moth super rainbow", "black sabbath", "black stone cherry", "black uhuru", "black veil brides", "blackfoot", "blake shelton", "blaze bayley", "bleeding through", "blind blake", "blind boy fuller", "blind guardian", "blind lemon jefferson", "blind willie mctell", "blindside", "blitzkid", "bloc party", "blondie", "blood on the dance floor", "blue cheer", "blue october", "blue rodeo", "blue stahli", "blue system", "blues traveler", "blutengel", "bo diddley", "boa", "boards of canada", "bob dylan", "bob rivers", "bob seger", "bobbie gentry", "bobby brown", "bobby darin", "bobby goldsboro", "bobby o", "bobby v", "bobby vee", "bobby vinton", "bobby womack", "body count", "bodyjar", "bolt thrower", "bon jovi", "bonfire", "bonnie raitt", "bonnie tyler", "borknagar", "boston", "bow wow", "boy george", "boyce avenue", "boysetsfire", "boyz ii men", "boyzone", "boz scaggs", "bracket", "brad paisley", "brainstorm", "brandi carlile", "brandy", "brazzaville", "bread", "breaking benjamin", "brenda lee", "brentalfloss", "brian eno", "brian hyland", "brian mcknight", "brian setzer", "brian wilson", "bride", "bright eyes", "bring me the horizon", "british sea power", "britney spears", "brockhampton", "broken social scene", "brook benton", "brotha lynch hung", "brownie mcghee", "bruce cockburn", "bruce dickinson", "bruce hornsby", "bruce springsteen", "bruno mars", "bryan adams", "bryan ferry", "bt", "bts", "buck 65", "buck owens", "buckcherry", "buddy guy", "buddy holly", "budgie", "building 429", "bukka white", "bullet for my valentine", "bumblefoot", "bun b", "bunny wailer", "burl ives", "burning heads", "burt bacharach", "burton cummings", "busta rhymes", "buzzcocks", "cab calloway", "caetano veloso", "cage", "calexico", "caliban", "calvin harris", "camel", "can", "cancerslug", "candlemass", "canibus", "canned heat", "cannibal corpse", "capercaillie", "capleton", "captain beefheart and the magic band", "car seat headrest", "caravan", "carbon leaf", "carcass", "carly simon", "carman", "carmen mcrae", "carole king", "carpenters", "carrie newcomer", "carrie underwood", "carter the unstoppable sex machine", "cass mccombs", "cassandra wilson", "casting crowns", "cat power", "cat stevens", "cathedral", "catie curtis", "cave in", "cece winans", "cee lo green", "celldweller", "celph titled", "celtic thunder", "celtic woman", "centinex", "chaka khan", "chamillionaire", "charles aznavour", "charles harrison", "charley pride", "charlie landsborough", "charlie louvin", "charlie musselwhite", "charlie peacock", "charlie rich", "charlotte martin", "cheap trick", "chenoa", "cher", "cheryl cole", "chet atkins", "chevelle", "chic", "chicago", "childish gambino", "children of bodom", "china crisis", "chris brown", "chris cornell", "chris de burgh", "chris isaak", "chris knight", "chris ledoux", "chris rea", "chris tomlin", "chris young", "christian bautista", "christina aguilera", "christoph eschenbach", "chubby checker", "chuck berry", "ciara", "cilla black", "circa survive", "circle jerks", "city and colour", "clan of xymox", "clannad", "clara haskil", "claude debussy", "claudio arrau", "clawfinger", "clay aiken", "clay walker", "cliff richard", "clint black", "closterkeller", "club 8", "cocorosie", "cocteau twins", "coil", "colbie caillat", "cold chisel", "colin blunstone", "colin hay", "colin james", "collective soul", "collin raye", "color me badd", "combichrist", "common", "con funk shun", "connie francis", "connie smith", "conor oberst", "converge", "conway twitty", "coolio", "corb lund", "corey hart", "corrosion of conformity", "countess", "country joe mcdonald", "covenant", "cowboy junkies", "cows", "cracker", "cradle of filth", "craig david", "creed", "creedence clearwater revival", "cross canadian ragweed", "crowded house", "cruachan", "crystal gayle", "crystal kay", "crystal lewis", "culture", "culture club", "current 93", "cursive", "curtis mayfield", "cypress hill", "d12", "daft punk", "dagoba", "damian marley", "damien jurado", "dan fogelberg", "dan seals", "dana winner", "dance gavin dance", "daniel lanois", "daniel lavoie", "daniel m ller schott", "daniel shafran", "danielson", "danko jones", "danzig", "dar williams", "darius rucker", "dark moor", "dark tranquillity", "darkest hour", "darkseed", "darkthrone", "darlene zschech", "daryl hall", "dashboard confessional", "dave alvin", "dave clark five", "dave davies", "dave dudley", "dave edmunds", "david and the giants", "david cassidy", "david crosby", "david crowder band", "david essex", "david geringas", "david knopfler", "david lee roth", "david meece", "david oistrakh", "david rovics", "david usher", "david wilcox", "dc talk", "de la soul", "dead can dance", "dead kennedys", "deadmau5", "dean martin", "deana carter", "death angel", "death cab for cutie", "death grips", "death in june", "debbie gibson", "debby boone", "deep purple", "deer tick", "deerhunter", "def leppard", "deftones", "degarmo and key", "deicide", "deine lakaien", "del shannon", "delbert mcclinton", "deliverance", "delta goodrem", "demi lovato", "demis roussos", "demon", "demon hunter", "deniece williams", "dennis brown", "descendents", "desmond dekker", "destroyer", "destruction", "dethklok", "devendra banhart", "devildriver", "devin townsend", "devin townsend project", "devo", "diamond rio", "diana krall", "diana ross", "dianne reeves", "diary of dreams", "dick haymes", "diddy", "dido", "die happy", "die krupps", "die toten hosen", "dimmu borgir", "dinah washington", "dino ciani", "dinu lipatti", "dio", "dion", "dionne warwick", "diorama", "dir en grey", "dire straits", "dirty heads", "disciple", "disclosure", "disturbed", "dixie chicks", "dizzee rascal", "dj shadow", "dmitri shostakovich", "dmx", "doc watson", "dog fashion disco", "dogwood", "dokken", "dolly parton", "don francisco", "don gibson", "don mclean", "don williams", "donna summer", "donovan", "dope", "doris day", "doro", "dottie west", "dougie maclean", "dover", "dragonforce", "drake", "dream evil", "dream theater", "dreamtale", "dropkick murphys", "drowning pool", "duke ellington", "dusty springfield", "dwight yoakam", "dying fetus", "eagles", "echolyn", "ed bruce", "ed harcourt", "ed sheeran", "eddi reader", "eddie from ohio", "eddie money", "eddie rabbitt", "eddy arnold", "eddy grant", "edenbridge", "edge of sanity", "edguy", "editors", "edvard grieg", "edwin fischer", "eighteen visions", "ektomorf", "elbow", "electric light orchestra", "electric six", "electric wizard", "eleni mandell", "elephant man", "elevation worship", "elf power", "elisa", "ella fitzgerald", "ella mae morse", "ellie goulding", "elliott smith", "ellis paul", "elmore james", "eloy", "elton john", "eluveitie", "elvenking", "elvis costello", "elvis presley", "elysian fields", "emil gilels", "emilie autumn", "eminem", "emmerson nogueira", "emmure", "emmylou harris", "en vogue", "enigma", "enrico caruso", "enrique iglesias", "ensiferum", "enslaved", "entombed", "epica", "epmd", "erasure", "eric bibb", "eric carmen", "eric church", "eric clapton", "eric johnson", "erick sermon", "ernest tubb", "erykah badu", "escape the fate", "esham", "esther ofarim", "ethel waters", "etta james", "eugenio finardi", "europe", "eurythmics", "eva cassidy", "evanescence", "evergrey", "every time i die", "everything but the girl", "evgeny kissin", "example", "exciter", "exodus", "extreme", "face to face", "fairport convention", "faith evans", "faith hill", "faith no more", "faithless", "falco", "falconer", "fall out boy", "falling up", "faron young", "fat joe", "fatboy slim", "fates warning", "fats domino", "fats waller", "faz l say", "fear factory", "felix mendelssohn bartholdy", "ferlin husky", "fernando ortega", "ffh", "field music", "fifteen", "fifth harmony", "fightstar", "firewind", "fish", "five finger death punch", "five iron frenzy", "flame", "fleetwood mac", "flo rida", "florida georgia line", "flying lotus", "flyleaf", "foetus", "foghat", "foreigner", "four tops", "fr d ric chopin", "francis poulenc", "franco battiato", "frank sinatra", "frank turner", "frank zappa", "frankie j", "frankie valli", "frankjavcee", "franz ferdinand", "franz liszt", "franz schubert", "freak kitchen", "fred astaire", "fred eaglesmith", "freddie hart", "freddie king", "freedom call", "frenzal rhomb", "frightened rabbit", "fritz kreisler", "front 242", "front line assembly", "fu manchu", "fugazi", "fun people", "funeral for a friend", "funkadelic", "funker vogt", "furry lewis", "fury in the slaughterhouse", "future", "gabrielle", "gaelic storm", "gaetano donizetti", "gaither vocal band", "galactic cowboys", "gamma ray", "gang starr", "garrick ohlsson", "garth brooks", "gary allan", "gary glitter", "gary moore", "gary numan", "gary stewart", "gene autry", "gene clark", "gene pitney", "gene vincent", "gene watson", "genesis", "gentle giant", "gentleman", "geoff moore", "georg philipp telemann", "george benson", "george clinton", "george formby", "george frideric handel", "george gershwin", "george harrison", "george jones", "george michael", "george strait", "georges bizet", "georges cziffra", "gerald levert", "gerald moore", "gerry rafferty", "geto boys", "ghostface killah", "giacomo puccini", "gidon kremer", "gil shaham", "gilberto gil", "gino vannelli", "ginuwine", "gioacchino rossini", "girls aloud", "girlschool", "giuseppe verdi", "glee cast", "glen campbell", "glenn frey", "glenn gould", "glenn hughes", "glenn miller", "gloria estefan", "gloria gaynor", "god module", "godsmack", "golden earring", "goldfinger", "goldfrapp", "good riddance", "goodie mob", "gordon bok", "gordon lightfoot", "gorillaz", "gotthard", "grace jones", "gramatik", "grand funk railroad", "grandaddy", "grateful dead", "grave", "grave digger", "graveworm", "great big sea", "great white", "greeley estates", "greg brown", "gregg allman", "gregor piatigorsky", "gregorian", "gregory isaacs", "grigory sokolov", "grimskunk", "grinspoon", "grits", "groundation", "guano apes", "guardian", "guiomar novaes", "gustav mahler", "guy clark", "guy mitchell", "guy sebastian", "gwar", "gwen stefani", "gym class heroes", "haemorrhage", "halestorm", "half man half biscuit", "halford", "hammerfall", "hank locklin", "hank snow", "hank thompson", "hank williams", "hank williams iii", "hanoi rocks", "hanson", "hanzel und gretyl", "hardcore superstar", "harem scarem", "harry belafonte", "harry chapin", "harry james", "harry nilsson", "haste the day", "hate", "hatebreed", "haujobb", "hawk nelson", "hawksley workman", "hawkwind", "hawthorne heights", "hayden", "hayley westenra", "heart", "heather alexander", "heather dale", "heather nova", "heaven shall burn", "heideroosjes", "heinrich schiff", "helloween", "helmet", "helstar", "henry mancini", "henryk szeryng", "herbie hancock", "herman brood", "hey", "hilary duff", "hillsong", "hilltop hoods", "hocico", "hoobastank", "hoodoo gurus", "hot chocolate", "hot dad", "hot water music", "house of lords", "howard jones", "howard shore", "huntingtons", "hypocrisy", "ian anderson", "ian hunter", "ian thomas", "ice cube", "iced earth", "icehouse", "iggy azalea", "iggy pop", "ignacy jan paderewski", "igor stravinsky", "ilse delange", "immortal technique", "impaled nazarene", "in flames", "in strict confidence", "in this moment", "incantation", "incubus", "indica", "indigo girls", "infected mushroom", "information society", "inna", "insane clown posse", "instalok", "iona brown", "iq", "iration", "irene cara", "iris dement", "iron fire", "iron maiden", "isaac hayes", "israel houghton", "israel vibration", "itzhak perlman", "ivan moravec", "izzy stradlin", "ja rule", "jack bruce", "jack greene", "jack johnson", "jack savoretti", "jack white", "jackie evancho", "jackie wilson", "jackson browne", "jag panzer", "jagged edge", "jake owen", "james blunt", "james brown", "james cotton", "james mcmurtry", "james taylor", "jamey johnson", "jamie cullum", "jandek", "jane monheit", "janet jackson", "janis ian", "janis joplin", "jann arden", "jars of clay", "jascha heifetz", "jason aldean", "jason derulo", "jason mraz", "jay sean", "jean michel jarre", "jean shepard", "jedi mind tricks", "jeff beck", "jeff healey", "jefferson airplane", "jefferson starship", "jennifer lopez", "jennifer rush", "jens lekman", "jeremy camp", "jermaine jackson", "jerry butler", "jerry jeff walker", "jerry lee lewis", "jerusalem", "jesse mccartney", "jessica simpson", "jessie j", "jesus culture", "jethro tull", "jewel", "jill scott", "jill sobule", "jim brickman", "jim croce", "jim ed brown", "jim jones", "jim reeves", "jimi hendrix", "jimmie rodgers", "jimmy buffett", "jimmy cliff", "jimmy dean", "jimmy dorsey", "jimmy reed", "jimmy rushing", "jls", "jo stafford", "joan armatrading", "joan baez", "joan jett and the blackhearts", "joan of arc", "joan osborne", "jodeci", "jody watley", "joe", "joe bonamassa", "joe budden", "joe cocker", "joe diffie", "joe ely", "joe henry", "joe jackson", "joe nichols", "joe purdy", "joe walsh", "johann sebastian bach", "johannes brahms", "john anderson", "john browning", "john denver", "john entwistle", "john farnham", "john fogerty", "john hammond", "john hiatt", "john lee hooker", "john legend", "john lennon", "john martyn", "john mayall", "john mayer", "john mccormack", "john mellencamp", "john michael montgomery", "john miles", "john ogdon", "john prine", "john stewart", "john waite", "john wesley harding", "john wetton", "johnny cash", "johnny hallyday", "johnny horton", "johnny mathis", "johnny mercer", "johnny paycheck", "johnny rivers", "johnny winter", "jojo", "jon anderson", "jon secada", "jonas brothers", "jonatha brooke", "joni mitchell", "jonny lang", "jordin sparks", "jorge bolet", "jorn", "josef suk", "joseph haydn", "joseph szigeti", "josh garrels", "josh groban", "josh ritter", "josh rouse", "josh turner", "josh woodward", "joshua bell", "joss stone", "journey", "joy division", "joy electric", "judas priest", "judie tzuke", "judy collins", "judy garland", "juelz santana", "juice newton", "julian lennon", "julian lloyd webber", "julie doiron", "julio iglesias", "jump5", "june carter cash", "jungle rot", "junior wells", "justin bieber", "justin timberlake", "justin townes earle", "kaiser chiefs", "kalmah", "kamelot", "kansas", "kanye west", "kari peitsamo", "karyn white", "kasey chambers", "kaskade", "kataklysm", "katatonia", "kate nash", "kate rusby", "katharine mcphee", "katherine jenkins", "katie melua", "katy perry", "kay starr", "keith sweat", "keith urban", "keith whitley", "kelis", "kelly clarkson", "kelly price", "ken hensley", "kendrick lamar", "kenny chesney", "kenny g", "kenny loggins", "kenny rogers", "kenny wayne shepherd", "kent", "keren ann", "keri hilson", "kid cudi", "kid rock", "kidz bop", "killing joke", "killswitch engage", "kim carnes", "kim mitchell", "kim wilde", "king crimson", "king diamond", "king missile", "kingdom come", "kirk franklin", "kiss", "kitty wells", "kmfdm", "koda kumi", "kodak black", "koko taylor", "kool keith", "korn", "korpiklaani", "kotoko", "kraan", "kraftwerk", "krayzie bone", "kreator", "kris kristofferson", "krisiun", "kristin hersh", "kt tunstall", "kula shaker", "kurupt", "kylie minogue", "la toya jackson", "labyrinth", "lacrimas profundere", "lacrimosa", "lacuna coil", "lady antebellum", "lady gaga", "ladytron", "lagwagon", "laibach", "lamb", "lamb of god", "lambchop", "lana del rey", "lana lane", "lang lang", "lara fabian", "larry norman", "laura branigan", "laura fygi", "laura marling", "laura pausini", "laurie anderson", "lazar berman", "lea salonga", "lead belly", "leatherface", "lecrae", "led zeppelin", "lee aaron", "lee greenwood", "lee hazlewood", "lefty frizzell", "leif ove andsnes", "lena horne", "lenny kravitz", "leo jan ek", "leo sayer", "leon fleisher", "leon russell", "leona lewis", "leonard bernstein", "leonard cohen", "leonard pennario", "leonid kogan", "lesley gore", "less than jake", "level 42", "levon helm", "lightning seeds", "lil boosie", "lil wayne", "lil yachty", "lily allen", "limp bizkit", "linda ronstadt", "lindisfarne", "linkin park", "linton kwesi johnson", "lionel hampton", "lionel richie", "lisa ekdahl", "lit", "lita ford", "little big town", "little mix", "little richard", "little river band", "little walter", "living colour", "living sacrifice", "ll cool j", "lmfao", "lobo", "logic", "lonestar", "lonnie donegan", "lonnie johnson", "lord belial", "lord of the lost", "lordi", "lords of acid", "loreena mckennitt", "loretta lynn", "lorrie morgan", "los lobos", "los pericos", "lou rawls", "lou reed", "loudon wainwright iii", "louis armstrong", "louis jordan", "louis lortie", "lucinda williams", "lucky dube", "ludacris", "ludwig van beethoven", "luka bloom", "luke bryan", "luna", "lupe fiasco", "luther allison", "luther vandross", "lyfe jennings", "lyle lovett", "lynn anderson", "lynyrd skynyrd", "m83", "ma rainey", "maanam", "maaya sakamoto", "mac davis", "mac miller", "macabre", "machinae supremacy", "machine gun kelly", "machine head", "macy gray", "mad caddies", "madeleine peyroux", "madness", "madonna", "maggie reilly", "magnum", "mai kuraki", "majesty", "malcolm holcombe", "malevolent creation", "malvina reynolds", "man", "mandy moore", "manfred mann", "manowar", "marc almond", "marc anthony", "marc cohn", "marcelle meyer", "marcia griffiths", "marduk", "maria jo o pires", "maria mena", "maria muldaur", "mariah carey", "marillion", "marilyn manson", "marina and the diamonds", "mario", "marissa nadler", "mark chesnutt", "mark erelli", "mark heard", "mark knopfler", "mark kozelek", "mark ronson", "marlene dietrich", "martha and the vandellas", "martha argerich", "martha wainwright", "martin carthy", "martina mcbride", "marty robbins", "marvin gaye", "mary mary", "mason jennings", "massive attack", "masta ace", "master p", "mastodon", "mat kearney", "matisyahu", "matt haimovitz", "matt maher", "matt nathanson", "matt pond pa", "matt redman", "matthew good", "matthew ryan", "maureen mcgovern", "maurice ravel", "maurizio pollini", "mavis staples", "max romeo", "mc chris", "mc eiht", "mc frontalot", "mc hammer", "mcfly", "me first and the gimme gimmes", "meat loaf", "meek mill", "megadeth", "mel tillis", "melanie", "melanie c", "melissa etheridge", "melissa manchester", "melvins", "memphis may fire", "memphis minnie", "memphis slim", "menudo", "mercyful fate", "mercyme", "merle haggard", "meshuggah", "metal church", "metallica", "method man", "metric", "mew", "mewithoutyou", "mgmt", "michael bolton", "michael card", "michael franks", "michael jackson", "michael johnson", "michael learns to rock", "michael martin murphey", "michael mcdonald", "michael nesmith", "michael rabin", "michael schenker group", "mickey gilley", "mickey newbury", "migos", "mika", "mike doughty", "mike jones", "mike oldfield", "mikhail pletnev", "mildred bailey", "miley cyrus", "millencolin", "mina", "mindless self indulgence", "ministry", "mint condition", "minus the bear", "mirah", "miranda lambert", "misanthrope", "misfits", "miss may i", "mississippi fred mcdowell", "mississippi john hurt", "missy elliott", "mobb deep", "moby", "modern talking", "modest petrovich mussorgsky", "molly hatchet", "monster magnet", "montgomery gentry", "monty python", "moonspell", "morbid angel", "morcheeba", "morgana lefay", "mormon tabernacle choir", "mortification", "mos def", "mose allison", "mostly autumn", "mother mother", "motorpsycho", "mott the hoople", "moya brennan", "mud", "muddy waters", "mudvayne", "municipal waste", "murray perahia", "murs", "mushroomhead", "musiq soulchild", "mustasch", "mxpx", "my life with the thrill kill kult", "my morning jacket", "myra hess", "mystic prophecy", "mystikal", "namie amuro", "nana mouskouri", "nanci griffith", "nancy sinatra", "nancy wilson", "napalm death", "nas", "nat king cole", "natalie cole", "natalie grant", "natalie imbruglia", "natasha bedingfield", "nathan milstein", "nazareth", "neal morse", "necro", "neil diamond", "neil sedaka", "neil young", "neko case", "nekromantix", "nektar", "nellie mckay", "nelly", "nelly furtado", "nelson", "nelson freire", "nena", "nerina pallot", "neurosis", "never shout never", "nevermore", "new edition", "new found glory", "new model army", "new order", "new riders of the purple sage", "new trolls", "new york philharmonic", "newsboys", "nic jones", "nick drake", "nick lowe", "nickelback", "nicki minaj", "nicole scherzinger", "night ranger", "nightrage", "nightwish", "nik kershaw", "nikka costa", "nile", "nina hagen", "nina simone", "nine inch nails", "ninja sex party", "nirvana", "nitty gritty dirt band", "no fun at all", "no use for a name", "noa", "nocturnal rites", "nofx", "nomeansno", "nonpoint", "norah jones", "norma jean", "nouvelle vague", "nox arcana", "nuclear assault", "nunslaughter", "obie trice", "odetta", "of montreal", "ohio players", "okkervil river", "olly murs", "omnia", "one direction", "opeth", "opus", "orchestral manoeuvres in the dark", "otep", "otis redding", "otis rush", "otis spann", "our last night", "outkast", "over the rhine", "overkill", "owl city", "ozzy osbourne", "pablo casals", "pain of salvation", "pam tillis", "pansy division", "pantera", "papa roach", "paradise lost", "paragon", "parkway drive", "parliament", "parov stelar", "parry gripp", "pascal rog ", "passenger", "pat benatar", "pathology", "patsy cline", "patti labelle", "patti page", "patty griffin", "patty loveless", "paul and storm", "paul anka", "paul carrack", "paul gilbert", "paul kelly", "paul mccartney", "paul oakenfold", "paul simon", "paul tortelier", "paul van dyk", "paul wall", "paul weller", "paul whiteman", "paul wilbur", "paul williams", "paul young", "peabo bryson", "peggy lee", "pendragon", "pennywise", "pentatonix", "pepper", "per gessle", "perry como", "pet shop boys", "pete seeger", "pete townshend", "peter and the test tube babies", "peter andre", "peter frampton", "peter gabriel", "peter green", "peter green splinter group", "peter hammill", "peter serkin", "peter tosh", "peter wolf", "petra", "petula clark", "phantom planet", "pharrell williams", "phil collins", "phil harris", "phil keaggy", "phil ochs", "phil wickham", "philadelphia orchestra", "philip glass", "philippe entremont", "piebald", "pierre fournier", "pierre laurent aimard", "pieter wispelwey", "pig", "pig destroyer", "pinback", "pink cream 69", "pink floyd", "pitbull", "pitchshifter", "planetshakers", "plumb", "poco", "poison", "polysics", "porter wagoner", "powerman 5000", "powerwolf", "prefab sprout", "pretenders", "primal fear", "primus", "prince", "procol harum", "project 86", "project pat", "project pitchfork", "prong", "propagandhi", "psyche", "psychostick", "public enemy", "puffy amiyumi", "puhdys", "pulp", "punchline", "pyotr ilyich tchaikovsky", "qntal", "quasi", "queen", "quiet riot", "racoon", "radu lupu", "raekwon", "raffi", "rage", "raimon", "ralph mctell", "ramones", "rancid", "randy newman", "randy rogers band", "randy stonehill", "randy travis", "rascal flatts", "rasputina", "ratt", "raul seixas", "ray charles", "ray davies", "ray lamontagne", "ray wylie hubbard", "rbd", "rebellion", "rebelution", "red flag", "reel big fish", "regina spektor", "rehab", "renaissance", "renard", "resurrection band", "reverend gary davis", "reverend horton heat", "revocation", "rhapsody of fire", "rheostatics", "rhonda vincent", "rich mullins", "richard goode", "richard strauss", "richard thompson", "richard wagner", "richie kotzen", "richie spice", "richmond fontaine", "rick astley", "rick ross", "rick springfield", "rick wakeman", "ricky martin", "ricky nelson", "ricky skaggs", "rihanna", "ringo starr", "riot", "rise against", "riverside", "robben ford", "robbie williams", "robert casadesus", "robert cray", "robert earl keen", "robert johnson", "robert palmer", "robert plant", "robert pollard", "robert schuman", "robert wyatt", "roberta flack", "robin trower", "robyn", "roch voisine", "rocky votolato", "rod stewart", "rodney crowell", "roger daltrey", "roger mcguinn", "roger miller", "roger waters", "roger whittaker", "rome", "ron sexsmith", "ronan keating", "ronnie milsap", "rory block", "rory gallagher", "rosalyn tureck", "rosanne cash", "rose royce", "rosemary clooney", "rotten sound", "rotting christ", "roxette", "roxy music", "roy acuff", "roy buchanan", "roy clark", "roy drusky", "roy orbison", "royal hunt", "rudimentary peni", "rudolf firku n ", "rudolf serkin", "rufus", "rufus wainwright", "running wild", "runrig", "rupaul", "rush", "ry cooder", "ryan adams", "rza", "s club 7", "sabaton", "sade", "saga", "sage francis", "said the whale", "saint etienne", "sam cooke", "sam phillips", "sam smith", "samael", "samantha fox", "sammie", "sammy hagar", "samson fran ois", "san francisco symphony", "sanctus real", "sandie shaw", "sandra", "sandy denny", "santana", "sara bareilles", "sara evans", "sarah brightman", "sarah connor", "sarah mclachlan", "sarah vaughan", "sasha", "savatage", "saves the day", "savoy brown", "sawyer brown", "saxon", "scarface", "schiller", "scooter", "scorpions", "scott joplin", "screaming trees", "screeching weasel", "seal", "sean paul", "sebadoh", "secret garden", "selena", "selena gomez", "senses fail", "sentenced", "sergei prokofiev", "sergei rachmaninoff", "sesame street", "sevendust", "seventh day slumber", "sex pistols", "shadows fall", "shaggy", "shakira", "shalamar", "sham 69", "shania twain", "shawn colvin", "shawn mendes", "shearwater", "sheena easton", "shel silverstein", "shelby lynne", "sheryl crow", "shinee", "shirley bassey", "shirley horn", "shlomo mintz", "shocking blue", "shooter jennings", "showaddywaddy", "showbread", "shura cherkassky", "sick of it all", "sigh", "silkk the shocker", "silverstein", "simon and garfunkel", "simple minds", "simply red", "sinister", "sinner", "sirenia", "sister hazel", "sister sledge", "six feet under", "skeeter davis", "skid row", "skinny puppy", "skip james", "skrillex", "skyclad", "skylark", "slade", "slayer", "slightly stoopid", "slim thug", "smokey robinson", "smokie", "smoking popes", "snfu", "snog", "snoop dogg", "sodom", "soft machine", "sol invictus", "solomon cutner", "son house", "son volt", "sonata arctica", "sondre lerche", "sonny boy williamson i", "sonny boy williamson ii", "sonny james", "sonny terry", "sophie zelmani", "sopor aeternus", "soulfly", "soulja boy", "spandau ballet", "sparks", "spice 1", "split enz", "spoon", "squeeze", "stacey kent", "stan ridgway", "stan rogers", "starflyer 59", "stars", "status quo", "steam powered giraffe", "steeleye span", "steely dan", "stephan eicher", "stephen malkmus", "stephen sondheim", "stephen stills", "steppenwolf", "steps", "stereolab", "steve earle", "steve forbert", "steve goodman", "steve green", "steve hackett", "steve miller band", "steve vai", "steve wariner", "steve winwood", "steven curtis chapman", "steven isserlis", "steven wilson", "stevie nicks", "stevie ray vaughan", "stevie wonder", "stick to your guns", "stiff little fingers", "sting", "stonewall jackson", "stratovarius", "strawbs", "stray cats", "stryper", "styles p", "styx", "sublime", "suburban legends", "sufjan stevens", "sugababes", "sugarland", "suicidal tendencies", "suicide commando", "suidakra", "sun kil moon", "super junior", "superbus", "supertramp", "survivor", "susan boyle", "suzanne vega", "suzi quatro", "suzy bogguss", "sviatoslav richter", "swans", "sweet", "sweetbox", "swollen members", "swv", "symphony x", "tad morose", "taj mahal", "take 6", "take that", "talib kweli", "talking heads", "tall dwarfs", "tammy wynette", "tampa red", "tanita tikaram", "tank", "tankard", "tanya tucker", "tarja", "tarot", "taylor dayne", "taylor swift", "team starkid", "tears for fears", "tech n9ne", "ted leo and the pharmacists", "ted nugent", "teena marie", "ten years after", "tennessee ernie ford", "tenth avenue north", "teresa brewer", "terminal choice", "terri clark", "terrorgruppe", "testament", "tex ritter", "tha dogg pound", "the 5th dimension", "the 69 eyes", "the acacia strain", "the adicts", "the alan parsons project", "the allman brothers band", "the ames brothers", "the amity affliction", "the andrews sisters", "the angels", "the animals", "the appleseed cast", "the arrogant worms", "the association", "the avett brothers", "the band", "the bangles", "the baseballs", "the beach boys", "the beatles", "the beau brummels", "the beautiful south", "the bellamy brothers", "the birthday massacre", "the black crowes", "the black dahlia murder", "the black eyed peas", "the black keys", "the bottle rockets", "the bouncing souls", "the boys", "the brian setzer orchestra", "the bronx", "the brothers four", "the browns", "the busters", "the byrds", "the cars", "the carter family", "the casualties", "the charlie daniels band", "the chemical brothers", "the chieftains", "the choir", "the chordettes", "the church", "the clash", "the clientele", "the corrs", "the coup", "the cramps", "the cross movement", "the crown", "the damned", "the darkness", "the derek trucks band", "the devil wears prada", "the dickies", "the dillinger escape plan", "the diplomats", "the divine comedy", "the donnas", "the doobie brothers", "the doors", "the drifters", "the dubliners", "the echoing green", "the everly brothers", "the fabulous thunderbirds", "the faint", "the felice brothers", "the fiery furnaces", "the fixx", "the flower kings", "the four aces", "the four lads", "the four seasons", "the frames", "the front bottoms", "the game", "the gathering", "the grass roots", "the guess who", "the handsome family", "the haunted", "the hellacopters", "the hold steady", "the hollies", "the human league", "the impressions", "the incredible string band", "the ink spots", "the irish rovers", "the isley brothers", "the jackson 5", "the jam", "the jayhawks", "the jimi hendrix experience", "the judds", "the kelly family", "the kills", "the kingston trio", "the kinks", "the kooks", "the lads", "the lawrence arms", "the locust", "the lucksmiths", "the magnetic fields", "the maine", "the manhattan transfer", "the manhattans", "the marvelettes", "the mavericks", "the mccalmans", "the meteors", "the mighty mighty bosstones", "the mission", "the moffatts", "the monkees", "the monochrome set", "the moody blues", "the muppets", "the national", "the new pornographers", "the nits", "the nylons", "the oak ridge boys", "the offspring", "the osmonds", "the outfield", "the partridge family", "the pillows", "the pineapple thief", "the platters", "the pogues", "the pointer sisters", "the police", "the poodles", "the posies", "the presidents of the united states of america", "the pretty things", "the proclaimers", "the prodigy", "the psychedelic furs", "the pussycat dolls", "the rascals", "the rasmus", "the raveonettes", "the real mckenzies", "the replacements", "the residents", "the righteous brothers", "the roches", "the rolling stones", "the roots", "the saints", "the samples", "the saturdays", "the saw doctors", "the script", "the searchers", "the seatbelts", "the seekers", "the shadows", "the shins", "the shirelles", "the smashing pumpkins", "the smithereens", "the smiths", "the soundtrack of our lives", "the spinners", "the stanley brothers", "the stone roses", "the stranglers", "the streets", "the string cheese incident", "the strokes", "the style council", "the stylistics", "the supremes", "the tea party", "the temptations", "the the", "the tragically hip", "the tremeloes", "the triffids", "the troggs", "the tubes", "the turtles", "the undertones", "the unseen", "the used", "the vandals", "the velvet underground", "the ventures", "the veronicas", "the verve", "the verve pipe", "the vines", "the violet burning", "the walkabouts", "the walker brothers", "the walkmen", "the wallflowers", "the wanted", "the waterboys", "the weeknd", "the whispers", "the white buffalo", "the white stripes", "the who", "the wiggles", "the wildhearts", "the wonder years", "the yardbirds", "the zombies", "thea gilmore", "theatre of tragedy", "theatres des vampires", "thee oh sees", "therion", "they might be giants", "thievery corporation", "thin lizzy", "third day", "third eye blind", "third world", "thompson twins", "thousand foot krutch", "three 6 mafia", "three dog night", "threshold", "thrice", "throwdown", "thunder", "thursday", "tiamat", "tim hughes", "tim maia", "tim mcgraw", "timbaland", "tina arena", "tina dico", "tina turner", "tinashe", "tindersticks", "tinie tempah", "tism", "tlc", "tnt", "toad the wet sprocket", "toby keith", "tobymac", "today is the day", "todd rundgren", "todd snider", "tokio hotel", "tom jones", "tom mcrae", "tom paxton", "tom petty and the heartbreakers", "tom rosenthal", "tom russell", "tom waits", "tommy dorsey", "tommy roe", "toni braxton", "tony banks", "tony bennett", "tony joe white", "tori amos", "toro y moi", "toto", "tourniquet", "townes van zandt", "toy dolls", "toyah", "tracy byrd", "tracy chapman", "tracy lawrence", "trae", "traffic", "trail of tears", "train", "transit", "transmetal", "trashcan sinatras", "travis", "travis scott", "trey songz", "trick daddy", "tricky", "trip lee", "tripod", "trisha yearwood", "tristania", "triumph", "trivium", "trooper", "trophy scars", "trouble", "tru", "truls m rk", "trust", "tsol", "tub ring", "twenty one pilots", "twila paris", "twista", "twisted sister", "twiztid", "ty segall", "type o negative", "tyrese", "tyrone wells", "u2", "ub40", "ufo", "ugk", "ultravox", "ulver", "uncle dave macon", "underoath", "underworld", "unkle", "unwritten law", "uriah heep", "useless id", "usher", "utada hikaru", "vader", "vampire rodents", "van cliburn", "van der graaf generator", "van halen", "van morrison", "vanden plas", "vanessa williams", "vangelis", "vanilla ice", "vast", "veggietales", "velvet acid christ", "venom", "vera lynn", "vern gosdin", "vertical horizon", "veruca salt", "vic chesnutt", "vic damone", "vicious rumors", "vienna philharmonic", "vigilantes of love", "viktoria mullova", "vince gill", "vinnie paz", "violent femmes", "virgin steele", "vision divine", "vladimir ashkenazy", "vladimir horowitz", "vladimir sofronitsky", "vnv nation", "voivod", "voltaire", "vonda shepard", "wale", "waltari", "walter gieseking", "walter trout", "wanda jackson", "war", "warrant", "warren zevon", "watch tower bible and tract society", "waylon jennings", "we the kings", "webb pierce", "webb wilder", "ween", "weezer", "weird al yankovic", "westlife", "whiskeytown", "white denim", "white heart", "whitechapel", "whitecross", "whitesnake", "whitney houston", "widespread panic", "wilco", "wilhelm backhaus", "wilhelm kempff", "will smith", "will young", "william fitzsimmons", "willie dixon", "willie nelson", "willy deville", "wilson phillips", "wilson pickett", "wings", "wire", "wise guys", "wishbone ash", "within temptation", "wizard", "wolfe tones", "wolfgang amadeus mozart", "wolfgang schneiderhan", "wolfgun", "woody guthrie", "wovenhand", "wumpscut", "wyclef jean", "wynn stewart", "wynonna judd", "x", "xandria", "xiu xiu", "xtc", "xv", "xxxtentacion", "xzibit", "yann tiersen", "yefim bronfman", "yehudi menuhin", "yello", "yellowcard", "yellowman", "yendri", "yes", "yngwie malmsteen", "yo la tengo", "yo yo ma", "yoko ono", "young jeezy", "young thug", "youri egorov", "zac brown band", "zachary richard"]
select_genre = "pop"#@param  ["No Preference", "Custom", "acoustic", "ambient", "americana", "bluegrass", "blues", "britpop", "celtic", "chanson", "christian", "christmas", "classical", "comedy", "country", "crossover", "dance", "deathcore", "deutschrock", "disco", "ebm", "electro", "electronic", "electronica", "electropop", "emo", "eurodance", "europop", "experimental", "fado", "folk", "funk", "gospel", "gothic", "grindcore", "grunge", "horrorcore", "house", "industrial", "jazz", "latin", "medieval", "metalcore", "movie", "mpb", "musical", "n", "neofolk", "pop", "psychedelic", "reggae", "reggaeton", "rock", "rockabilly", "salsa", "samba", "schlager", "sertanejo", "ska", "soul", "soundtrack", "swing", "synthpop", "tejano", "television", "trance", "vocal", "worship"]
custom_artist = "" #@param {type:"string"}
custom_genre = "" #@param {type:"string"}

if select_artist == "No Preference":
  select_artist = "unknown"
if select_genre == "No Preference":
  select_genre = "unknown"

if select_artist == "Custom":
  select_artist = custom_artist
if select_genre == "Custom":
  select_genre = custom_genre

sampling_temperature = 1.0 #@param {type:"number"}

#@markdown ---

#@markdown Continuous Repriming Settings:

reprime_sampling_mode = "new code" #@param ["old code", "new code"]

timing_mode = 'bpm' #@param ['tokens', 'seconds', 'bpm']

#@markdown Token/Seconds Mode

amount = 0 #@param {type: "integer"}

#@markdown BPM Mode

bpm = 105 #@param {type: "number"}
bars = 4 #@param {type: "integer"}
beats = 0 #@param {type: "integer"}

beats += bars * 4

if gpu_info[0].find('Tesla T4') >= 0:
  lower_batch_size = 10
  print('Tesla T4 detected, lower_batch_size set to 10')
elif gpu_info[0].find('Tesla K80') >= 0:
  lower_batch_size = 8
  print('Tesla K80 detected, lower_batch_size set to 8')
elif gpu_info[0].find('Tesla P100') >= 0:
  lower_batch_size = 16
  print('Tesla P100 detected, lower_batch_size set to 16')
elif gpu_info[0].find('Tesla V100') >= 0:
  lower_batch_size = 16
  print('Tesla V100 detected, lower_batch_size set to 16')
elif gpu_info[0].find('A100') >= 0:
  lower_batch_size = 30
  print('A100 detected! Party Time!!! lower_batch_size set to 30')
else:
  lower_batch_size = 9
  print('Different GPU detected, lower_batch_size set to 9')
lower_level_chunk_size = 32

if reprime_sampling_mode == "old code":
  try:
    top_prior
  except:
    top_prior = None

  if top_prior:
    top_prior.prior.transformer.del_cache()
    first = True
    while zs[2].shape[1] < duration // top_prior.raw_to_tokens:

      #amnt_to_sample = int(hps.hop_fraction[2] * top_prior.n_ctx)
      if timing_mode == 'tokens':
        amnt_to_sample = amount
        do_speed = False # broken atm
      elif timing_mode == 'seconds':
        seconds_done = round(zs[2].shape[1] / (hps.sr / top_prior.raw_to_tokens))
        seconds_done += amount
        amnt_to_sample = round(seconds_done * (hps.sr / top_prior.raw_to_tokens)) - zs[2].shape[1]
        do_speed = False # broken atm(44100*amount/top_prior.raw_to_tokens) % 1.0 == 0
      elif timing_mode == 'bpm':
        beats_done = round((zs[2].shape[1] / (hps.sr / top_prior.raw_to_tokens)) / (60 / bpm))
        beats_done += beats
        amnt_to_sample = round(beats_done * (60 / bpm) * (hps.sr / top_prior.raw_to_tokens)) - zs[2].shape[1]
        do_speed = False # broken atm(44100*60*beats/bpm/top_prior.raw_to_tokens) % 1.0 == 0

      assert amnt_to_sample < top_prior.n_ctx, 'too long sections m8'
      
      amnt_to_encode = top_prior.n_ctx - amnt_to_sample
      start = zs[2].shape[1] - amnt_to_encode
      metas = [dict(artist = select_artist,
                  genre = select_genre,
                  total_length = duration,
                  offset = max(0, start) * top_prior.raw_to_tokens,
                  lyrics = your_lyrics,
                  ),
                ] * hps.n_samples
      labels = top_prior.labeller.get_batch_labels(metas, 'cuda')

      if first:
        print('can speed sample:', do_speed)

      if start >= 0 and do_speed and hps.n_samples == 1 and duration / top_prior.raw_to_tokens >= start + amnt_to_sample * i + top_prior.n_ctx:
        print('speed sampling...')
        print(zs[2].shape[1])
        aud = t.zeros((max_batch_size, top_prior.n_ctx * top_prior.raw_to_tokens, 1), dtype=t.float16)
        for i in range(max_batch_size):
          aud[i] = raw_audio[0, (start + amnt_to_sample * i) * top_prior.raw_to_tokens : (start + amnt_to_sample * i + top_prior.n_ctx) * top_prior.raw_to_tokens]
        xs = vqvae.encode(aud, bs_chunks=max_batch_size)
        xs[-1] = xs[-1][:,:-amnt_to_sample]

        sampling_kwargs = dict(temp=sampling_temperature, fp16=True, max_batch_size=max_batch_size, chunk_size=chunk_size)
        
        metass = (metas * max_batch_size).copy()
        for i in range(max_batch_size):
          metass[i]['offset'] = metass[i]['offset'] + (amnt_to_sample * i * top_prior.raw_to_tokens)
        labelss = top_prior.labeller.get_batch_labels(metass, 'cuda')
        xs=sample_partial_window(xs, labelss, sampling_kwargs, 2, top_prior, amnt_to_sample, hps)
        zs[2] = t.cat((zs[2], xs[2][:,-amnt_to_sample:].reshape(1,-1)), dim=1)
      else:
        if zs[2].shape[1] > 0:
          enc = raw_audio[:, max(0, start * top_prior.raw_to_tokens) : (start + top_prior.n_ctx) * top_prior.raw_to_tokens]
          xs = vqvae.encode(enc, bs_chunks=raw_audio.shape[0])
          xs[-1] = xs[-1][:,:-amnt_to_sample]
        else:
          xs = [t.zeros(hps.n_samples,0,dtype=t.long, device='cpu') for _ in range(hps.levels)]
        sampling_kwargs = dict(temp=sampling_temperature, fp16=True, max_batch_size=max_batch_size, chunk_size=chunk_size)

        xs=sample_partial_window(xs, labels, sampling_kwargs, 2, top_prior, amnt_to_sample, hps, autosave=False)
          
        zs[2] = t.cat((zs[2], xs[2][:,-amnt_to_sample:]), dim=1)
      t.save(zs, f'{hps.name}/tokens.t')
      x = vqvae.decode(zs[2:], start_level=2).cpu().numpy()
      for i in range(hps.n_samples):
        sf.write(f'{hps.name}/level_2/item_{i}.wav', x[i][:], hps.sr)
        print(f'WAV written to disk.')
      first = False

  metas = [dict(artist = select_artist,
                genre = select_genre,
                total_length = duration,
                offset = 0,
                lyrics = your_lyrics,
                ), ] * hps.n_samples

if reprime_sampling_mode == "new code":
  try:
    top_prior
  except:
    top_prior = None
    
  if top_prior:
    top_prior.prior.transformer.del_cache()
    first = True
    while zs[2].shape[1] < duration // top_prior.raw_to_tokens:
      left_to_sample = duration // top_prior.raw_to_tokens - zs[2].shape[1]

      amnt_to_sample = []
      zs_size_projected = zs[2].shape[1]
      for i in range(max_batch_size // hps.n_samples):
        
        if timing_mode == 'tokens':
          new_tokens = amount
        elif timing_mode == 'seconds':
          seconds_done = round(zs_size_projected / (hps.sr / top_prior.raw_to_tokens))
          seconds_done += amount
          new_tokens = round(seconds_done * (hps.sr / top_prior.raw_to_tokens)) - zs_size_projected
        elif timing_mode == 'bpm':
          beats_done = round((zs_size_projected / (hps.sr / top_prior.raw_to_tokens)) / (60 / bpm))
          beats_done += beats
          new_tokens = round(beats_done * (60 / bpm) * (hps.sr / top_prior.raw_to_tokens)) - zs_size_projected
          
        assert new_tokens < top_prior.n_ctx, 'too long sections m8'

        amnt_to_sample.append(min(new_tokens, left_to_sample))
        left_to_sample -= new_tokens
        zs_size_projected += new_tokens
        if left_to_sample <= 0:
          break

      print('amnt_to_sample:', amnt_to_sample)
      max_to_sample = max(amnt_to_sample)
      #print('max_to_sample:', max_to_sample)
      print(zs[2].shape[1], '/', duration // top_prior.raw_to_tokens, '| aprox windows left:', round((duration // top_prior.raw_to_tokens - zs[2].shape[1]) / max_to_sample, 1))
      metas = []
      xs_pre_cat = []

      zs_size_projected = zs[2].shape[1]
      for i in range(len(amnt_to_sample)):
        amnt_to_encode = top_prior.n_ctx - max_to_sample
        start = zs_size_projected - amnt_to_encode
        metas.extend([dict(
            artist = select_artist,
            genre = select_genre,
            total_length = duration,
            offset = max(0, start) * top_prior.raw_to_tokens,
            lyrics = your_lyrics,
            )] * hps.n_samples
        )
      
        if zs_size_projected > 0:
          enc = raw_audio[:, max(0, start * top_prior.raw_to_tokens) : (start + top_prior.n_ctx) * top_prior.raw_to_tokens]
          xs = vqvae.encode(enc, bs_chunks=raw_audio.shape[0])
          #x = vqvae.decode(xs[2:], start_level=2).cpu().numpy()
          #for i in range(hps.n_samples):
          #  librosa.output.write_wav(f'{hps.name}top_level_encdec_{i}.wav', x[i][:], sr=hps.sr)

          #print('xs:', xs[2].shape)
          xss = xs[-1][:,:-max_to_sample]
          #print('xss:', xss.shape)
          #xss = xss[:, max(0,top_prior.n_ctx - (xss.shape[1] + max_to_sample)):]
          #print('xss:', xss.shape)
        else:
          xss = t.zeros(hps.n_samples,0,dtype=t.long, device='cpu')

        xs_pre_cat.append(xss)
        zs_size_projected += amnt_to_sample[i]
        
      labels = top_prior.labeller.get_batch_labels(metas, 'cuda')

      #print('metas:', metas)
      #print('labels:', labels)

      mx = max(map(lambda x: x.shape[1], xs_pre_cat))
      #print('mx:', mx)
      xs_pre_cat = list(map(lambda x: F.pad(x, (mx - x.shape[1], 0), mode='constant', value=0), xs_pre_cat))

      #print('xs_pre_cat:', list(map(lambda x: x.shape, xs_pre_cat)))

      xs = [
        t.zeros(hps.n_samples,0,dtype=t.long, device='cpu'),
        t.zeros(hps.n_samples,0,dtype=t.long, device='cpu'),
        t.cat(xs_pre_cat, dim=0)
      ]

      #print(xs[2].shape)
      #print(max_to_sample)
      sampling_kwargs = dict(temp=sampling_temperature, fp16=True, max_batch_size=max_batch_size, chunk_size=chunk_size)
      
      xs=sample_partial_window(xs, labels, sampling_kwargs, 2, top_prior, max_to_sample, hps, autosave=False)

      foobar = []
      for i in range(len(amnt_to_sample)):
        foobar.append(xs[2][i*hps.n_samples:(i+1)*hps.n_samples,-max_to_sample:][:,:amnt_to_sample[i]])
      #print('foobar:', foobar)
      zs[2] = t.cat((zs[2], t.cat(foobar, dim=1)), dim=1)
      t.save(zs, f'{hps.name}/tokens.t')
      x = vqvae.decode(zs[2:], start_level=2).cpu().numpy()
      for i in range(hps.n_samples):
        sf.write(f'{hps.name}/level_2/item_{i}.wav', x[i][:], hps.sr)
        print(f'WAV written to disk.')
      first = False

  metas = [dict(artist = select_artist,
                genre = select_genre,
                total_length = duration,
                offset = 0,
                lyrics = your_lyrics,
                ), ] * hps.n_samples

import numpy as np
#zs[2] = zs[2][:,:duration//128]
x = vqvae.decode(zs[2:], start_level=2).cpu().numpy()
for i in range(hps.n_samples):
  sf.write(f'{hps.name}/level_2/item_{i}.wav', x[i][:], hps.sr)
  print(f'WAV written to disk. Look for your noisy render at: {hps.name}/level_2/item_{i}.wav')
if reprime_sampling_mode == "new code":
  sf.write(f'{hps.name}/level_2/item_combined.wav', (np.sum(x,axis=0) + raw_audio[0,:x.shape[1]].numpy()/2)/(1+hps.n_samples), hps.sr)
  print(f"WAV written to disk. If your bpm is correct, look for the compare render at: {hps.name}/level_2/item_combined.wav")
del x


# Set this False if you are on a local machine that has enough memory (this allows you to do the
# lyrics alignment visualization during the upsampling stage). For a hosted runtime, 
# we'll need to go ahead and delete the top_prior if you are using the 5b_lyrics model.
if True:
  del top_prior
  empty_cache()
  top_prior=None
upsamplers = [make_prior(setup_hparams(prior, dict()), vqvae, 'cpu') for prior in priors[:-1]]
sampling_kwargs = [dict(temp=0.99, fp16=True, max_batch_size=lower_batch_size, chunk_size=lower_level_chunk_size),
                    dict(temp=0.99, fp16=True, max_batch_size=lower_batch_size, chunk_size=lower_level_chunk_size),
                    None]
labels = [prior.labeller.get_batch_labels(metas, 'cuda') for prior in upsamplers] + [upsamplers[0].labeller.get_batch_labels(metas, 'cuda')]

for prior in upsamplers:
  prior.prior.transformer.del_cache()
empty_cache()
zs = upsample(zs, labels, sampling_kwargs, [*upsamplers, top_prior], hps)
print("The Continuous Reprime is complete! Look for your render at: "+hps.name+"/level_0/item_0.wav")

Mounted at /content/gdrive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/craftmine1000/jukebox-saveopt.git to /tmp/pip-req-build-x0yve8a5
  Running command git clone --filter=blob:none --quiet https://github.com/craftmine1000/jukebox-saveopt.git /tmp/pip-req-build-x0yve8a5
  Resolved https://github.com/craftmine1000/jukebox-saveopt.git to commit 5b76e9e07eb15dba6fd79da99d76d1ecb32a7ea5
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 KB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 KB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 47.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for jukebox: filename=jukebox-1.0-py3-none-any.whl size=

<ipython-input-3-faf0e6a87158>:171: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  primer_length_in_seconds = librosa.get_duration(filename=audio_file)


Running  wget -O /root/.cache/jukebox/models/5b/vqvae.pth.tar https://openaipublic.azureedge.net/jukebox/models/5b/vqvae.pth.tar
Restored from /root/.cache/jukebox/models/5b/vqvae.pth.tar
0: Loading vqvae in eval mode
Loading artist IDs from /usr/local/lib/python3.9/dist-packages/jukebox/data/ids/v2_artist_ids.txt
Loading artist IDs from /usr/local/lib/python3.9/dist-packages/jukebox/data/ids/v2_genre_ids.txt
Level:2, Cond downsample:None, Raw to tokens:128, Sample length:1048576
Running  wget -O /root/.cache/jukebox/models/5b_lyrics/prior_level_2.pth.tar https://openaipublic.azureedge.net/jukebox/models/5b_lyrics/prior_level_2.pth.tar
Restored from /root/.cache/jukebox/models/5b_lyrics/prior_level_2.pth.tar
0: Loading prior in eval mode
Tesla T4 detected, lower_batch_size set to 10
amnt_to_sample: [3150, 3150]
0 / 61425 | aprox windows left: 19.5
Sampling 6300 tokens for [0,6300]. Conditioning on 3150 tokens
Primed sampling 2 samples with temp=1.0, top_k=0, top_p=0.0
25/25 [0:00:05<0:

# Guide to the above settings:

**your_lyrics:** Specify the lyrics Jukebox should attempt to follow. You can paste any lyrics you want in here or leave it blank, which will result in gibberish.

**model:**
OpenAI has trained a few different models for Jukebox. In this notebook, you can access the 5b_lyrics, 5b and 1b_lyrics models. As you can imagine, the 5b_lyrics model is the superior one, but also requires a stronger GPU to run properly. Which model you should choose depends on the GPU you were assigned, which you can check in the first cell of the notebook. Recommended settings: 5b_lyrics on P100 or T4 GPU, 1b_lyrics on K80 GPU.
(5b_lyrics theoretically works on a K80 now, but sampling is going to be super slow.)

(5b is like 5b_lyrics, without supporting custom lyrics, so it will generate gibberish vocals)

**hps.name:** Specifies the name of the folder in Google Drive, where you will find your results in. Make sure to choose a different name for each of your runs, or else the notebook will get confused.

**speed_upsampling:** If selected, will upsample much faster, at the cost of the samples sounding slightly "choppy".

**audio_file:** Specifies which song Jukebox will generate a new audio. Upload the file you want (can be WAV, FLAC, MP3, M4A, OPUS, etc.) to the root directory of your Google Drive and fill in its name above.

**sampling_temperature:** Determines the creativity and energy of Jukebox. The higher the temperature, the more chaotic and intense the result will be. You can experiment with this. Recommended to keep between .96 and .999

List of the v2 (5b_lyrics & 5b models) & v3 (1b_lyrics model) artist and genre ids: https://github.com/openai/jukebox/tree/master/jukebox/data/ids

Important links:

Official blog: https://openai.com/blog/jukebox/
Original repo: https://github.com/openai/jukebox/

License: Non-commercial, for details see: https://github.com/openai/jukebox/blob/master/LICENSE

This notebook was created by: Jaime v2.0 - Since 2018. (https://www.youtube.com/channel/UCWbk5lrnDGB6SnhnIwcDZ4w)

If you have any bugs and/or typos, please contact me at: jaimev2.#4985.

# Changelog

2022-09-26 - locked hps.n_samples to 1 due to avoid memory crashes and also now auto-writing .wav file while the level_2 is sampling. (Requested by Erwill#0590) 